In [19]:
# Import libraries
import cv2
import dlib
import numpy as np
import json
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from face_encodings import Base, Face  # Import your database model

In [20]:
# Database connection
engine = create_engine('sqlite:///face_encodings.db')
Session = sessionmaker(bind=engine)
session = Session()

In [21]:
face_rec_model = dlib.face_recognition_model_v1(r'C:\Users\adam.bloebaum\.vscode\driveline\cv\dlib_face_recognition_resnet_model_v1.dat')
cnn_face_detector = dlib.cnn_face_detection_model_v1(r'C:\Users\adam.bloebaum\.vscode\driveline\cv\mmod_human_face_detector.dat')
shape_predictor = dlib.shape_predictor(r"C:\Users\adam.bloebaum\.vscode\driveline\cv\shape_predictor_68_face_landmarks.dat")

In [22]:
def load_known_encodings(session):
    known_encodings = []
    # If you have names or IDs, also load them
    known_names = []
    faces = session.query(Face).all()  # Modify according to your database schema

    for face in faces:
        # Load and deserialize the encoding
        known_encodings.append(json.loads(face.encoding))
        known_names.append(face.id)

    return known_encodings, known_names

known_encodings, known_names = load_known_encodings(session)


In [23]:
# extract every Nth frame from the video
def extract_frames(video_path, frame_skip=50):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:
            frames.append(frame)
        frame_count += 1

    cap.release()
    return frames

In [24]:
# detect faces
def detect_faces(frames, cnn_face_detector):
    detected_faces = []

    for frame in frames:
        # convert the OpenCV BGR image to RGB
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        faces = cnn_face_detector(rgb_image, 1)

        for face in faces:
            x, y, w, h = face.rect.left(), face.rect.top(), face.rect.width(), face.rect.height()
            detected_faces.append(rgb_image[y:y+h, x:x+w])

    return detected_faces

In [25]:
def extract_face_encodings(face):
    # convert the OpenCV BGR image to RGB
    rgb_image = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

    # initialize the Dlib face detector and shape predictor
    face_detector = dlib.get_frontal_face_detector()

    # detect faces
    detected_faces = face_detector(rgb_image, 1)
    if len(detected_faces) == 0:
        # no faces found in the image, return None or an appropriate value
        return None

    # get the landmarks/parts for the face
    shape = shape_predictor(rgb_image, detected_faces[0])

    # align and crop the face using the landmarks
    face_chip = dlib.get_face_chip(rgb_image, shape)

    # compute the face descriptor
    face_descriptor = face_rec_model.compute_face_descriptor(face_chip)

    # convert the descriptor to a numpy array, then to a list, and serialize to JSON
    encoding = np.array(face_descriptor).tolist()
    return json.dumps(encoding)

In [26]:
def identify_faces_in_video(video_path, known_encodings, known_names):
    frames = extract_frames(video_path)
    for frame in frames:
        detected_faces = detect_faces([frame], cnn_face_detector)
        for face in detected_faces:
            face_encoding_json = extract_face_encodings(face)
            if face_encoding_json is not None:
                encoding = json.loads(face_encoding_json)

                # Calculate distances to known encodings
                distances = np.linalg.norm([encoding] - known_encodings, axis=1)
                best_match_index = np.argmin(distances)

                if distances[best_match_index] < 0.6:
                    name = known_names[best_match_index]
                    print(f"Identified: {name}")
                else:
                    print("Unknown face detected")
            else:
                print("No face found in this frame.")


In [27]:
identify_faces_in_video(r"C:\Users\adam.bloebaum\.vscode\driveline\cv\MoCap 91.0 mph Nov-02-2023.mp4", known_encodings, known_names)

No face found in this frame.
No face found in this frame.
No face found in this frame.
No face found in this frame.
No face found in this frame.
No face found in this frame.
No face found in this frame.
No face found in this frame.
No face found in this frame.
No face found in this frame.
No face found in this frame.
